In [ ]:
!pip install sentence-transformers

In [8]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker import KMeans
from sagemaker.predictor import Predictor
from sagemaker.model import Model
from sagemaker import image_uris
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.decomposition import TruncatedSVD
import pandas as pd

In [2]:
# SageMaker 클라이언트 및 세션 설정
sagemaker_session = sagemaker.Session()
role = get_execution_role()
s3_bucket = 'dothis-ai-test'  # 모델 아카이브가 저장된 S3 버킷 이름
s3_prefix = 'models/kmeans-test-1-2023-06-23-02-51-53-846'  # 모델 아카이브가 저장된 S3 경로 접두사
# 모델 아카이브의 S3 경로
model_s3_path = f's3://{s3_bucket}/{s3_prefix}/output/model.tar.gz'

print(model_s3_path)

s3://dothis-ai-test/models/kmeans-test-1-2023-06-23-02-51-53-846/output/model.tar.gz


In [12]:
# SageMaker 엔드포인트 생성을 위한 설정
endpoint_name = 'kmeans-test-1'  # 엔드포인트에 할당할 모델 이름
endpoint_config_name = 'kmeans-test-1-config'
instance_type = 'ml.m5.xlarge'  # 엔드포인트 인스턴스 유형
initial_instance_count = 1  # 엔드포인트 인스턴스 수

In [4]:
# K-Means 모델 로드
model_uri = image_uris.retrieve('kmeans', sagemaker_session.boto_region_name)
kmeans_model = Model(model_data=model_s3_path, role=role, image_uri=model_uri)

In [13]:
from datetime import datetime

endpoint_name = f"{endpoint_name}-{datetime.utcnow():%Y-%m-%d-%H%M}"
print("EndpointName =", endpoint_name)

EndpointName = kmeans-test-1-2023-06-23-0445


In [15]:
# 모델 배포하기
kmeans_model.deploy(
                    initial_instance_count=initial_instance_count,
                    instance_type=instance_type,
                    endpoint_name=endpoint_name
)

-----!

In [2]:
# S3에 액세스할 수 있는 클라이언트 생성
s3 = boto3.client('s3')
bucket = 'dothis-ai-test'
data_key = 'data/video_data_20230414.txt'

# S3에서 파일 가져오기
response = s3.get_object(Bucket=bucket, Key=data_key)

# 파일 내용 읽기
file_content = response['Body'].read().decode('utf-8')

# 파일 내용 출력
train_data = file_content[1:].split("\r\n")[:200]
test_data = file_content[1:].split("\r\n")[200:300]

In [3]:
# SentenceTransformer을 사용한 문장 임베딩
embedding_name = "jhgan"
sentence_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [4]:
# vectors = [sentence_model.encode(vector) for vector in tqdm(train_data)]
vectors = [sentence_model.encode(vector) for vector in test_data]
sentence_vectors = np.vstack(vectors)

In [6]:
n_components = 100
svd = TruncatedSVD(n_components=n_components)
test_lsa_sentence_vectors = svd.fit_transform(sentence_vectors)
test_lsa_sentence_vectors.shape

(100, 100)

In [9]:
# 입력 데이터를 CSV 형식으로 변환
input_data = pd.DataFrame(test_lsa_sentence_vectors).astype('float32').to_csv(header=False, index=False)

In [16]:
# 추론 요청
endpoint_name = "kmeans-test-1-2023-06-23-0445"
runtime_client = boto3.client('sagemaker-runtime')
response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=input_data)
response

{'ResponseMetadata': {'RequestId': '8d7cb1a7-514a-4041-b3ec-980060eb83e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8d7cb1a7-514a-4041-b3ec-980060eb83e8',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Fri, 23 Jun 2023 05:33:27 GMT',
   'content-type': 'application/json',
   'content-length': '6824'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7f1c84f5da50>}

In [18]:
# 추론 결과 파싱
import ast

result = response['Body'].read().decode('utf-8')

# 문자열을 딕셔너리로 변환
dictionary = ast.literal_eval(result)
result = dictionary["predictions"]



Cluster labels: [{'closest_cluster': 0.0, 'distance_to_cluster': 7.626099586486816}, {'closest_cluster': 0.0, 'distance_to_cluster': 7.662365436553955}, {'closest_cluster': 8.0, 'distance_to_cluster': 7.857890605926514}, {'closest_cluster': 7.0, 'distance_to_cluster': 9.355652809143066}, {'closest_cluster': 4.0, 'distance_to_cluster': 10.251152992248535}, {'closest_cluster': 0.0, 'distance_to_cluster': 10.641250610351562}, {'closest_cluster': 4.0, 'distance_to_cluster': 10.139921188354492}, {'closest_cluster': 1.0, 'distance_to_cluster': 9.41203498840332}, {'closest_cluster': 8.0, 'distance_to_cluster': 9.751779556274414}, {'closest_cluster': 0.0, 'distance_to_cluster': 10.151679039001465}, {'closest_cluster': 8.0, 'distance_to_cluster': 8.920455932617188}, {'closest_cluster': 8.0, 'distance_to_cluster': 8.406658172607422}, {'closest_cluster': 4.0, 'distance_to_cluster': 9.492510795593262}, {'closest_cluster': 8.0, 'distance_to_cluster': 7.850658416748047}, {'closest_cluster': 0.0, 'di

In [24]:
# 클러스터링 결과 출력

for i in range(len(test_data)):
    print(test_data[i], int(result[i]["closest_cluster"]))

당혹 예고 귀신 헬리콥터 매입 도시 곳곳 메시지 비밀 SBS SBS시사교양 달리 달리보라 SBS달리 스브스 교양프로그램 당신이혹하는사이 당혹사 음모론 음모론토크쇼 토크쇼 솔깃한식탁 윤종신 장진 봉태규 변영주 송은이 곽재식 완벽한타인 꼬꼬무 꼬리에꼬리를무는이야기 혹하는이야기 도시괴담 이야기꾼 스토리텔링 시나리오 장기밀매 장기매매 귀신헬리콥터 0
정도 수라상 광주 EP OLIFE 채널 일요일 skyTV 스카이티비 오라이프채널 OLIFE OLIFE채널 오라이프 낭만비박 집단가출 집단가출 낭만비박 티비조선 TV조선 허영만 신현준 류시원 ENA 여행예능 여행 낭만 HCN 충북방송 MGTV 다문화TV 버킷리스트 낭만비박 허영만 신현준 류시원 배기성 광주 0
경제 전쟁 거래 토큰 증권 생태 자본  돌풍  찻잔 태풍 곽수종 인호 정상호 김세희 한국경제TV 경제 증권 방송 투자전략 종목상담 한경TV 와우넷 주식창 TMI특공대 돈립만세 글로벌마켓 부동산 월스트리트 경제전쟁꾼 꾼 경제전쟁꾼시즌3 생방송 곽수종 인호 정상호 김세희 토큰 토큰증권 증권 STO ST NFT 조각투자 금융자산 블록체인 증권사 수혜주 투자 증시 증권시장 한국경제TV 와우넷 와우파 주식창 한경KVINA 파트너 경제 증권 와우 WOWTV 8
구글 검색 클라우드 게임 XBOX XBOX클라우드게이밍 클라우드게이밍 구글 지포스 지포스나우 아마존 루나 구글 스태디아 google amazon luna google stadia geforce  헤일로인피니트 halo infinite 유정군 XBOX게임패스 XBOX게임패스얼티밋 Xboxgamepass XBOX게임패스얼티밋 클라우드게이밍 구글서비스 7
여자  욕실 줄눈 셀프 시공  시공  변기 백시멘트 변기 바닥 보수 욕실 줄눈 보수 더러운 변기 백시멘트 셀프 시공 욕실줄눈 시공 욕실 줄눈 변기 교체 줄눈 셀프 시공 백시멘트 화장실 줄눈 화장실 줄눈 셀프 4
다이어트 VLOG 야근 다이어트 kg 안녕 다이어트  아가히어터 복부지방흡입 복부지흡 지방흡입 지방흡입브이로그 다이어

In [25]:
# SageMaker 클라이언트 생성
sagemaker_client = boto3.client('sagemaker')

# 엔드포인트 삭제 요청
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'b597179b-2169-4d36-a5d9-634baf1074d4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b597179b-2169-4d36-a5d9-634baf1074d4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Fri, 23 Jun 2023 05:36:52 GMT'},
  'RetryAttempts': 0}}